In [1]:
def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = []
        for line in f:
            src, tgt = line.strip().split("\t")
            data.append((src, tgt))
        return data
    
data =load_data("D:\\Users\\user\\git\\practice\\data\\en_kr.txt")
print(data)

[('Hello, how are you?', '안녕하세요, 어떻게 지내세요?'), ('What is your name?', '당신의 이름은 무엇인가요?'), ('I’m fine, thank you.', '잘 지내요, 감사합니다.'), ('Where are you from?', '어디에서 오셨나요?'), ('Please sit down.', '앉아주세요.'), ('Don’t touch that!', '그것을 만지지 마세요!'), ('Could you help me, please?', '도와주실 수 있나요?'), ('What time is it?', '지금 몇 시인가요?'), ('It’s 3 o’clock.', '3시입니다.'), ('How much does it cost?', '가격이 얼마인가요?'), ('It costs ten dollars.', '10달러입니다.'), ('Where is the nearest subway station?', '가장 가까운 지하철역은 어디인가요?'), ('I would like to go to the airport.', '공항으로 가고 싶어요.'), ('Do you have a map?', '지도 있나요?'), ('I’d like a coffee, please.', '커피 한 잔 주세요.'), ('Can I see the menu?', '메뉴를 볼 수 있을까요?'), ('Is this dish spicy?', '이 요리는 매운가요?'), ('I need to go to the supermarket.', '나는 슈퍼마켓에 가야 해요.'), ('The weather is nice today.', '오늘 날씨가 좋네요.'), ('I don’t feel well.', '몸이 좋지 않아요.'), ('I woke up at 7 AM.', '나는 오전 7시에 일어났어요.'), ('What are you doing this weekend?', '이번 주말에 뭐 하세요?'), ('See you tomorrow.', '내일 봐요.'), ('I a

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(data, tokenizer, max_length=128):
    tokenized_data = []
    for src,tgt in data:
        src_tokens = tokenizer(src, max_length=max_length, padding = "max_length", truncation=True, return_tensors="pt")
        tgt_tokens = tokenizer(tgt, max_length=max_length, padding = "max_length", truncation=True, return_tensors="pt")
        tokenized_data.append((src_tokens, tgt_tokens))
    return tokenized_data

tokenized_data = tokenize_data(data, tokenizer)

In [3]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src, tgt = self.data[idx]
        return {
            "input_ids": src["input_ids"].squeeze(0),
            "attention_mask": src["attention_mask"].squeeze(0),
            "labels": tgt["input_ids"].squeeze(0),
        }
    
dataset = TextDataset(tokenized_data)

In [4]:
print(len(tokenized_data))

26


In [5]:
from torch.utils.data import DataLoader

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# DataLoader 사용 예제
for batch in dataloader:
    print(batch["input_ids"].shape)  # (batch_size, seq_len)
    print(batch["labels"].shape)    # (batch_size, seq_len)

torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 128])


In [6]:
# 고유 단어 수 계산
english_words = set()
korean_words = set()
for src, tgt in data:
    english_words.update(src.split())
    korean_words.update(tgt.split())

print(f"English unique words: {len(english_words)}")
print(f"Korean unique words: {len(korean_words)}")

English unique words: 88
Korean unique words: 69


In [7]:
# 데이터셋에서 가장 긴 문장의 길이 계산
max_length = max(entry["input_ids"].shape[0] for entry in dataset)

# 여유 값을 더해 position 설정
position = max_length + 10  # 여유로 10 추가
print(f"Max length: {max_length}, Position: {position}")


Max length: 128, Position: 138


In [8]:
vocab_size=tokenizer.vocab_size
d_model=256 
position=138 
num_heads=4
num_encoder_layers = 6
num_decoder_layers = 6
num_epochs = 10

In [9]:
from Transformer import Transformer
transformer_model = Transformer(
    vocab_size, 
    d_model, 
    position, 
    num_heads, 
    num_encoder_layers, 
    num_decoder_layers)

In [10]:
total_params = sum(p.numel() for p in transformer_model.parameters())
print(f"Total Parameters: {total_params}")

Total Parameters: 34925370


In [11]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer_model.parameters(), lr = 0.001)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model = transformer_model.to(device)
loss_fn = loss_fn.to(device)
transformer_model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0

    for i, data in enumerate(dataloader):
        # 데이터를 GPU로 이동
        inputs = data['input_ids'].to(device)
        labels = data['labels'].to(device)
        attention_mask = data['attention_mask'].to(device)

        # Optimizer gradient 초기화
        optimizer.zero_grad()

        # 모델 출력
        outputs = transformer_model(inputs, labels, attention_mask)

        # 차원 조정
        outputs = outputs.view(-1, outputs.size(-1))
        labels = labels.view(-1)

        # 손실 계산
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Optimizer 업데이트
        optimizer.step()

        # 손실 값 누적
        running_loss += loss.item()

        # 1000번째 배치마다 손실 출력
        if i % 1000 == 999:
            batch_loss = running_loss / 1000
            print(f"  Batch {i+1}: Loss = {batch_loss:.4f}")
            running_loss = 0.0

    # 에포크 종료 시 평균 손실 출력
    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1} completed. Average Loss: {epoch_loss:.4f}")


Epoch 1/10
Epoch 1 completed. Average Loss: 2.7294
Epoch 2/10
Epoch 2 completed. Average Loss: 1.1366
Epoch 3/10
Epoch 3 completed. Average Loss: 0.9430
Epoch 4/10
Epoch 4 completed. Average Loss: 0.9115
Epoch 5/10
Epoch 5 completed. Average Loss: 0.9036
Epoch 6/10
Epoch 6 completed. Average Loss: 0.9046
Epoch 7/10
Epoch 7 completed. Average Loss: 0.9023
Epoch 8/10
Epoch 8 completed. Average Loss: 0.8989
Epoch 9/10
Epoch 9 completed. Average Loss: 0.9054
Epoch 10/10
Epoch 10 completed. Average Loss: 0.9018


In [38]:
eng = "Hello, how are you?"

input = tokenizer(eng, max_length=max_length, padding = "max_length", truncation=True, return_tensors="pt")
output_ids = [tokenizer.cls_token_id]
transformer_model.eval()


for i in range(max_length):
    # output_ids를 입력 길이에 맞추어 패딩 추가
    current_output_len = len(output_ids)
    if current_output_len < input['input_ids'].size(1):  # 입력 길이보다 짧으면
        padded_output_ids = output_ids + [tokenizer.pad_token_id] * (input['input_ids'].size(1) - current_output_len)
    else:
        padded_output_ids = output_ids

    tgt_mask = torch.triu(torch.ones(max_length, max_length), diagonal=1).bool()
    tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0).to(device)
    
    logit = transformer_model(input['input_ids'].to(device), torch.tensor(padded_output_ids, device=device).unsqueeze(0), tgt_mask)

    probs = torch.nn.functional.softmax(logit[:,-1,:], dim=-1)

    next_token = torch.argmax(probs, dim=-1).item()

    if next_token == tokenizer.eos_token_id:
        break

    output_ids.append(next_token)

krn = tokenizer.decode(output_ids, skip_special_tokens=True)

In [39]:
krn

''